# Import

In [18]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

## Add configuration file

In [19]:
sys.path.append("/home/jovyan/core/config/")

In [20]:
from ALL import config 

## Set condition

In [21]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

# Read data

In [22]:
ag_news_df = pd.read_csv("../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [23]:
with open("../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [24]:
documents = [
    TaggedDocument(doc.split(" "), [i]) for i, doc in enumerate(ag_news_df.words)
]

In [25]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]

In [26]:
def get_doc2vec(texts, vector_size,seed, path):
    model = Doc2Vec(
        documents,
        vector_size=vector_size,
        window=5,
        min_count=1,
        epochs=50,
        seed=seed,
        workers=os.cpu_count(),
    )
    model.save(path)
    vectors = [model.infer_vector(text) for text in texts]
    return vectors

In [27]:
model_nums = 1

In [28]:
vectors_path = "../data/AgNews/doc2vec/vector/"
models_path = "../data/AgNews/doc2vec/model/"
for vector_size in tqdm(vector_dims):
    for model_num in [2, 3, 4, 5]:
        vector_path = f"{vectors_path}/{vector_size}/"
        model_path = f"{models_path}/{vector_size}/"
        os.makedirs(vector_path, exist_ok=True)
        os.makedirs(model_path, exist_ok=True)

        vectors = get_doc2vec(
            ag_news_df.words.apply(lambda word: word.split(" ")),
            vector_size,
            seed=model_num,
            path=f"{model_path}{model_num}.model",
        )

        np.savetxt(
            f"{vector_path}{model_num}.csv",
            np.stack(vectors),
            delimiter=",",
        )

 57%|█████▋    | 8/14 [10:21:33<7:46:10, 4661.72s/it]


OSError: [Errno 28] No space left on device